In [21]:
#test
import sagemaker
import boto3
from sagemaker import get_execution_role

import pandas as pd
import numpy as np
import sys

sys.path.extend(['../src', '../config'])

region = boto3.Session().region_name
session = sagemaker.Session()
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

import helpers.instance as ins
import helpers.s3 as s3_helper
import helpers.utils as ut
import helpers.athena as at

# READ YAML FILES AND STORE RELEVANT PATHS
dataset_cfg = ins.read_config('../config/datasets.yaml') 
config_cfg = ins.read_config('../config/config.yaml')
sql_cfg = ins.read_config('../config/sql.yaml')

In [28]:
# passing Athena variables
database=['ptledw_playarea', 'ptl_maintenance', 'ptl_connfleet', 'proactive_maint_db', 'ptl_adhoc']
workgroup='WG-Advanalytics'
tablename=['d_location_master','constellation_rules','ufch_depup', 'archive_pd_inference_scenario_output', 'corp_foilanl']
#query_string = 'select * from ptl_maintenance.constellation_rules'
query_string = 'select * from {}.{}'.format(database[0],tablename[0])
# 'select * from ptledw_playarea.d_location_master'

In [29]:
query_string

'select * from ptledw_playarea.d_location_master'

## Read from Athena

In [31]:
# read data from Athena
df = at.get_data_from_athena(region=region,  query_string=query_string, 
                                      database=database[0],
                                      workgroup=workgroup)

Checking query status
Current status: QUEUED
Current status: RUNNING
Current status: RUNNING
Current status: SUCCEEDED


## Persist as a parquet

In [32]:
path = ['s3://pske-stg-advanalytics/Projects/Unit_Sale_Risk_Interns/Data/Interim/location_master/']
s3_helper.persist_file_to_path(df=df,
                       path=path[0],
                       filetype='parquet')

## Read from Parquet

In [33]:
df2 = s3_helper.read_parquet_from_path(path=path[0])
df2

,location_master_key,corp_cd,corp_nm,corp_address1,corp_address2,corp_city,corp_state_code,corp_state_name,corp_zip,corp_county,...,rdist_maint_cost_class,parent_dist_sufx,location_original_key,dist_overhead_type,onsiteshop_flag,auto_svc_net_loc_flg,exists_fdstmst_flag,day_light_saving_flag,time_zone_cd,locn_temp_closed
0,2633,HPTL,PENSKE TRUCK LEASING CO.,2675 MORGANTOWN RD,,READING,PA,PENNSYLVANIA,19607,BERKS,...,5,15,2633,HIGH,None,,Y,Y,D,N
1,6512,1690,PENSKE SERVICIOS DE ADM,CARRETERA MONTEREY,LAREDO KILOMETER #20,"APODACA, NUEVO LEON",DF,DIST. FEDERAL,65542,,...,,10,6512,None,None,None,Y,Y,C,N
2,10577,HPTL,PENSKE TRUCK LEASING CO.,RT 10 GREEN HILLS,,READING,PA,PENNSYLVANIA,19607,BERKS,...,4,10,1547,STANDARD,None,None,Y,Y,E,N
3,9052,HPTL,PENSKE TRUCK LEASING CO.,2675 MORGANTOWN RD,,READING,PA,PENNSYLVANIA,19607,BERKS,...,6,12,2337,HIGH,None,,Y,Y,D,N
4,5066,3000,PENSKE LOGISTICS INC,3401 ENTERPRISE PARKWAY,SUITE 200,BEACHWOOD,OH,OHIO,44122,CUYAHOGA,...,,79,5066,None,None,None,Y,Y,D,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12657,5864,3000,PENSKE LOGISTICS INC,3401 ENTERPRISE PARKWAY,SUITE 200,BEACHWOOD,OH,OHIO,44122,CUYAHOGA,...,,00,5864,None,None,None,Y,Y,D,N
12658,5553,3000,PENSKE LOGISTICS INC,3401 ENTERPRISE PARKWAY,SUITE 200,BEACHWOOD,OH,OHIO,44122,CUYAHOGA,...,,52,5553,None,None,None,Y,Y,D,N
12659,9450,2000,PTL CANADA INC,1610 ENTERPRISE ROAD,,MISSISSAUGA,ON,ONTARIO,L4W 4L4,ONTARIO,...,,,9450,None,None,,Y,Y,E,N
12660,4389,3000,PENSKE LOGISTICS INC,3401 ENTERPRISE PARKWAY,SUITE 200,BEACHWOOD,OH,OHIO,44122,CUYAHOGA,...,,84,4389,None,None,None,Y,Y,D,N
